In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score # cross_val_score ekledim
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
import math
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn import decomposition, datasets

In [2]:
data_full = pd.read_csv("A.csv")
data = data_full[["product_id","sales", "price", "basket", "fav", "visit", "impression", "quantity", "quantity_demand", "remove_from_fav", "reviewCount", "rating"]]

In [3]:
dateData = pd.read_csv("specialDates.csv")
for column in dateData.columns:
    data[column]=np.tile(dateData[column].to_numpy(),len(data_full["product_id"].unique())) 

<ipython-input-3-574209a0da70>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column]=np.tile(dateData[column].to_numpy(),len(data_full["product_id"].unique()))


In [4]:
def lasso(data):
    WholeLasso = []
    for product in data["product_id"].unique():
        #Productlasso = {}
        productLasso=[]
        prodData = data[data.product_id == product]
        prodData["price"]=prodData["price"].shift(7)
        prodData["basket"]=prodData["basket"].shift(7)
        prodData["fav"]=prodData["fav"].shift(7)
        prodData["visit"]=prodData["visit"].shift(7)
        prodData["impression"]=prodData["impression"].shift(7)
        prodData["quantity"]=prodData["quantity"].shift(7)
        prodData["quantity_demand"]=prodData["quantity_demand"].shift(7)
        prodData["remove_from_fav"]=prodData["remove_from_fav"].shift(7)
        prodData["reviewCount"]=prodData["reviewCount"].shift(7)
        prodData["rating"]=prodData["rating"].shift(7)

        X = prodData[["price", "basket", "fav", "visit", "impression", "quantity", "quantity_demand", "remove_from_fav", "reviewCount", "rating","haftasonu","kampanya","resmitatil","ozelGun","Ozel3","Ozel5"]][7:]
        y = prodData["sales"][7:]
        X_train = X[:math.floor(4*200/5)]
        X_test = X[math.floor(4*200/5):]
        y_train = y[:math.floor(4*200/5)]
        y_test = y[math.floor(4*200/5):]

        pipeline = Pipeline([
                         ('scaler',StandardScaler()),
                         ('model',Lasso())
        ])

        #Searching for best alpha
        search = GridSearchCV(pipeline,
                          {'model__alpha':np.arange(1e-7,1e-5,1e-7)}, # burayı nasıl değiştireceğimi anlamadım tam
                          cv = 5, scoring="neg_mean_squared_error",verbose=3
                          )
        #fit model
        search.fit(X_train,y_train)
        
        coefficients = search.best_estimator_.named_steps['model'].coef_
        #print(search.best_estimator_,"AAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
        
        alph= 9.9e-06 # üsttekine göre değiştirdim 
        lasso= Lasso(alpha=alph)
        lasso.fit(X_train, y_train)
        y_train_pred= lasso.predict(X_train)
        y_test_pred= lasso.predict(X_test)
        productLasso.append(mean_absolute_error(y_train,y_train_pred))
        productLasso.append(mean_absolute_error(y_test,y_test_pred))
        importance = np.abs(coefficients)
        productLasso=np.concatenate((importance, np.array(productLasso)), axis=None)
        #print(productLasso)
        WholeLasso.append(productLasso)
    return WholeLasso
        
    
    
    

In [5]:
labels = ["trainMAE","testMAE","price", "basket", "fav", "visit", "impression", "quantity", "quantity_demand", "remove_from_fav", "reviewCount", "rating","haftasonu","kampanya","resmitatil","ozelGun","Ozel3","Ozel5"]
WholeLasso=lasso(data)
WholeLasso=pd.DataFrame(WholeLasso, columns=labels)
WholeLasso

<ipython-input-4-d28a01f2d812>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prodData["price"]=prodData["price"].shift(7)
<ipython-input-4-d28a01f2d812>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prodData["basket"]=prodData["basket"].shift(7)
<ipython-input-4-d28a01f2d812>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END .............................model__alpha=1e-07; total time=   0.0s
[CV 2/5] END .............................model__alpha=1e-07; total time=   0.0s
[CV 3/5] END .............................model__alpha=1e-07; total time=   0.0s
[CV 4/5] END .............................model__alpha=1e-07; total time=   0.0s
[CV 5/5] END .............................model__alpha=1e-07; total time=   0.0s
[CV 1/5] END .............................model__alpha=2e-07; total time=   0.0s
[CV 2/5] END .............................model__alpha=2e-07; total time=   0.0s
[CV 3/5] END .............................model__alpha=2e-07; total time=   0.0s
[CV 4/5] END .............................model__alpha=2e-07; total time=   0.0s
[CV 5/5] END .............................model__alpha=2e-07; total time=   0.0s
[CV 1/5] END .............................model__alpha=3e-07; total time=   0.0s
[CV 2/5] END .............................model

[CV 4/5] END ...........................model__alpha=2.4e-06; total time=   0.0s
[CV 5/5] END ...........................model__alpha=2.4e-06; total time=   0.0s
[CV 1/5] END ............model__alpha=2.4999999999999998e-06; total time=   0.0s
[CV 2/5] END ............model__alpha=2.4999999999999998e-06; total time=   0.0s
[CV 3/5] END ............model__alpha=2.4999999999999998e-06; total time=   0.0s
[CV 4/5] END ............model__alpha=2.4999999999999998e-06; total time=   0.0s
[CV 5/5] END ............model__alpha=2.4999999999999998e-06; total time=   0.0s
[CV 1/5] END ............model__alpha=2.5999999999999997e-06; total time=   0.0s
[CV 2/5] END ............model__alpha=2.5999999999999997e-06; total time=   0.0s
[CV 3/5] END ............model__alpha=2.5999999999999997e-06; total time=   0.0s
[CV 4/5] END ............model__alpha=2.5999999999999997e-06; total time=   0.0s
[CV 5/5] END ............model__alpha=2.5999999999999997e-06; total time=   0.0s
[CV 1/5] END ............mod

[CV 2/5] END ...........................model__alpha=4.5e-06; total time=   0.0s
[CV 3/5] END ...........................model__alpha=4.5e-06; total time=   0.0s
[CV 4/5] END ...........................model__alpha=4.5e-06; total time=   0.0s
[CV 5/5] END ...........................model__alpha=4.5e-06; total time=   0.0s
[CV 1/5] END ...........................model__alpha=4.6e-06; total time=   0.0s
[CV 2/5] END ...........................model__alpha=4.6e-06; total time=   0.0s
[CV 3/5] END ...........................model__alpha=4.6e-06; total time=   0.0s
[CV 4/5] END ...........................model__alpha=4.6e-06; total time=   0.0s
[CV 5/5] END ...........................model__alpha=4.6e-06; total time=   0.0s
[CV 1/5] END ...........................model__alpha=4.7e-06; total time=   0.0s
[CV 2/5] END ...........................model__alpha=4.7e-06; total time=   0.0s
[CV 3/5] END ...........................model__alpha=4.7e-06; total time=   0.0s
[CV 4/5] END ...............

[CV 1/5] END .............model__alpha=6.699999999999999e-06; total time=   0.0s
[CV 2/5] END .............model__alpha=6.699999999999999e-06; total time=   0.0s
[CV 3/5] END .............model__alpha=6.699999999999999e-06; total time=   0.0s
[CV 4/5] END .............model__alpha=6.699999999999999e-06; total time=   0.0s
[CV 5/5] END .............model__alpha=6.699999999999999e-06; total time=   0.0s
[CV 1/5] END .............model__alpha=6.799999999999999e-06; total time=   0.0s
[CV 2/5] END .............model__alpha=6.799999999999999e-06; total time=   0.0s
[CV 3/5] END .............model__alpha=6.799999999999999e-06; total time=   0.0s
[CV 4/5] END .............model__alpha=6.799999999999999e-06; total time=   0.0s
[CV 5/5] END .............model__alpha=6.799999999999999e-06; total time=   0.0s
[CV 1/5] END .............model__alpha=6.899999999999999e-06; total time=   0.0s
[CV 2/5] END .............model__alpha=6.899999999999999e-06; total time=   0.0s
[CV 3/5] END .............mo

[CV 1/5] END .............................model__alpha=9e-06; total time=   0.0s
[CV 2/5] END .............................model__alpha=9e-06; total time=   0.0s
[CV 3/5] END .............................model__alpha=9e-06; total time=   0.0s
[CV 4/5] END .............................model__alpha=9e-06; total time=   0.0s
[CV 5/5] END .............................model__alpha=9e-06; total time=   0.0s
[CV 1/5] END .............model__alpha=9.100000000000001e-06; total time=   0.0s
[CV 2/5] END .............model__alpha=9.100000000000001e-06; total time=   0.0s
[CV 3/5] END .............model__alpha=9.100000000000001e-06; total time=   0.0s
[CV 4/5] END .............model__alpha=9.100000000000001e-06; total time=   0.0s
[CV 5/5] END .............model__alpha=9.100000000000001e-06; total time=   0.0s
[CV 1/5] END ...........................model__alpha=9.2e-06; total time=   0.0s
[CV 2/5] END ...........................model__alpha=9.2e-06; total time=   0.0s
[CV 3/5] END ...............

,trainMAE,testMAE,price,basket,fav,visit,impression,quantity,quantity_demand,remove_from_fav,reviewCount,rating,haftasonu,kampanya,resmitatil,ozelGun,Ozel3,Ozel5
0,83.180528,14.205738,47.368766,194.970376,30.033393,0.0,0.0,271.107268,84.476328,133.604032,47.649166,11.898218,17.366016,101.964898,143.915932,33.793435,278.258955,279.456693


In [6]:
filename="A-Lasso.csv"
WholeLasso.to_csv(filename, sep=',',header=True)